In [8]:
import geopandas as gpd
import pandas as pd
import csv

In [ ]:
# Load shapefile
shapefile_path = 'wards_poly.shp'
gdf = gpd.read_file(shapefile_path)

# Replace & with and
gdf['wd17nm'] = gdf['wd17nm'].str.replace('&', 'and')

# Load CSV file
csv_path = pd.read_csv('land-registry-house-prices-ward.csv')

csv_path['Year'] = csv_path['Year'].astype(str)
value_2017 = csv_path[csv_path['Year'].str.endswith('2017')]

# Remove rows where Value column contains '-'
value_2017_cleaned = value_2017[value_2017['Value'] != '-']

# Remove commas and convert to integer
value_2017_cleaned['Value'] = value_2017_cleaned['Value'].str.replace(',', '').astype(int)


print(f"Number of rows after cleaning: {len(value_2017_cleaned)}")
print(value_2017_cleaned.head())


Number of rows after cleaning: 7548
            Code       Ward_name               Borough                  Year  \
53551  E05000026           Abbey  Barking and Dagenham  Year ending Mar 2017   
53552  E05000027          Alibon  Barking and Dagenham  Year ending Mar 2017   
53553  E05000028       Becontree  Barking and Dagenham  Year ending Mar 2017   
53554  E05000029  Chadwell Heath  Barking and Dagenham  Year ending Mar 2017   
53555  E05000030       Eastbrook  Barking and Dagenham  Year ending Mar 2017   

      Measure   Value  
53551  Median  230000  
53552  Median  295000  
53553  Median  300000  
53554  Median  300000  
53555  Median  330000  


C:\Users\wendy\AppData\Local\Temp\ipykernel_11500\326070850.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  value_2017_cleaned['Value'] = value_2017_cleaned['Value'].str.replace(',', '').astype(int)


In [ ]:
ward_names_2017_check = value_2017_cleaned[value_2017_cleaned['Ward_name'].str.contains("Margarets")]

# Get same names for wards that don't match
value_2017_cleaned["Ward_name"] = value_2017_cleaned["Ward_name"].replace(
    {
        "St Margarets and North Twickenha": "St Margarets and North Twickenham",
        "St Helen's": "St. Helen's",
        "Ham, Petersham and Richmond Rive": "Ham, Petersham and Richmond Riverside"
    },
    regex=False  # Ensures exact matching instead of treating it as a regex pattern
)

#print(ward_names_2017cleaned_check)

             Code                         Ward_name               Borough  \
54054   E05000527  St Margarets and North Twickenha  Richmond upon Thames   
54684   E05000527  St Margarets and North Twickenha  Richmond upon Thames   
55314   E05000527  St Margarets and North Twickenha  Richmond upon Thames   
55944   E05000527  St Margarets and North Twickenha  Richmond upon Thames   
110124  E05000527  St Margarets and North Twickenha  Richmond upon Thames   
110754  E05000527  St Margarets and North Twickenha  Richmond upon Thames   
111384  E05000527  St Margarets and North Twickenha  Richmond upon Thames   
112014  E05000527  St Margarets and North Twickenha  Richmond upon Thames   
166194  E05000527  St Margarets and North Twickenha  Richmond upon Thames   
166824  E05000527  St Margarets and North Twickenha  Richmond upon Thames   
167454  E05000527  St Margarets and North Twickenha  Richmond upon Thames   
168084  E05000527  St Margarets and North Twickenha  Richmond upon Thames   

C:\Users\wendy\AppData\Local\Temp\ipykernel_11500\19648027.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  value_2017_cleaned["Ward_name"] = value_2017_cleaned["Ward_name"].replace(


In [ ]:
# Only keep one year section to keep unique values
value_2017_check = value_2017_cleaned[value_2017_cleaned['Year'].str.contains('Year ending Mar 2017')]

# Only keep median values
value_2017_median = value_2017_check[value_2017_check['Measure'].str.contains('Median')]

print(value_2017_median)


['Abbey' 'Alibon' 'Becontree' 'Chadwell Heath' 'Eastbrook' 'Eastbury'
 'Gascoigne' 'Goresbrook' 'Heath' 'Longbridge' 'Mayesbrook' 'Parsloes'
 'River' 'Thames' 'Valence' 'Village' 'Whalebone' 'Brunswick Park'
 'Burnt Oak' 'Childs Hill' 'Colindale' 'Coppetts' 'East Barnet'
 'East Finchley' 'Edgware' 'Finchley Church End' 'Garden Suburb'
 'Golders Green' 'Hale' 'Hendon' 'High Barnet' 'Mill Hill' 'Oakleigh'
 'Totteridge' 'Underhill' 'West Finchley' 'West Hendon' 'Woodhouse'
 'Barnehurst' 'Belvedere' 'Blackfen and Lamorbey' 'Blendon and Penhill'
 'Brampton' 'Christchurch' 'Colyers' 'Crayford' 'Cray Meadows'
 'Danson Park' 'East Wickham' 'Erith' 'Falconwood and Welling'
 'Lesnes Abbey' 'Longlands' 'North End' 'Northumberland Heath' "St Mary's"
 "St Michael's" 'Sidcup' 'Thamesmead East' 'Alperton' 'Barnhill'
 'Brondesbury Park' 'Dollis Hill' 'Dudden Hill' 'Fryent' 'Harlesden'
 'Kensal Green' 'Kenton' 'Kilburn' 'Mapesbury' 'Northwick Park' 'Preston'
 'Queens Park' 'Queensbury' 'Stonebridge' 'S

In [ ]:
# Merge on the ward codes and names
wards_median_price_merged = gdf.merge(
    value_2017_median,
    left_on=['wd17nm', 'wd17cd'],
    right_on=['Ward_name', 'Code'],
    how='inner'
)

# Save result
#wards_median_price_merged.to_file('London_House_Prices_2017_adjusted.shp')

# Print final merged DataFrame
print(wards_median_price_merged)

     OBJECTID     wd17cd          wd17nm wd17nmw   bng_e   bng_n      long  \
0         1.0  E05000026           Abbey    None  544434  184378  0.081291   
1         2.0  E05000027          Alibon    None  549247  185196  0.150987   
2         3.0  E05000028       Becontree    None  546863  185869  0.116912   
3         4.0  E05000029  Chadwell Heath    None  548266  189340  0.138596   
4         5.0  E05000030       Eastbrook    None  550774  186272  0.173453   
..        ...        ...             ...     ...     ...     ...       ...   
624    5779.0  E05009401    Queen's Gate    None  526009  179281 -0.186050   
625    5780.0  E05009402       Redcliffe    None  525937  177919 -0.187580   
626    5781.0  E05009403  Royal Hospital    None  527849  178310 -0.159910   
627    5782.0  E05009404     St. Helen's    None  523717  181499 -0.218280   
628    5783.0  E05009405         Stanley    None  526743  177955 -0.175960   

           lat                                GlobalID     Code